In [6]:
import numpy as np  
from collections import OrderedDict

from ConfigParser import ConfigParser

#cfgfile = '/data/darknet/cfg/yolov3-tiny.cfg'
#weights = '/data/darknet/weights/yolov3-tiny.weights'
cfgfile = '/data/github-repos/darknet/ncs/yolov3-tiny-ncs-test.cfg'
weights = '/data/github-repos/darknet/ncs/yolov3-tiny-ncs_20000.weights'

class uniqdict(OrderedDict):
    _unique = 0
    def __setitem__(self, key, val):
        if isinstance(val, OrderedDict):
            self._unique += 1
            key += "_"+str(self._unique)
        OrderedDict.__setitem__(self, key, val)
parser = ConfigParser(dict_type=uniqdict)
parser.read(cfgfile)
blocks = parser.sections()
print(blocks)
fp = open(weights, "rb")
#### The first 4 values are header information   
# 1. Major version number  
# 2. Minor Version Number  
# 3. Subversion number   
# 4. IMages seen   
header = np.fromfile(fp, dtype = np.int32, count = 5)  
print(header)
#fp = open(weightfile, 'rb')  
#header = np.fromfile(fp, count=5, dtype=np.int32)  
#header = np.ndarray(shape=(5,),dtype='int32',buffer=fp.read(20))  
#print(header)  
buf = np.fromfile(fp, dtype = np.float32)
#np.savetxt('aa.txt', buf[7743340:])

fp.close()
#print(buf)
start = 0
pre_filters = 3
for block in blocks:
        print(block) 
        items = dict(parser.items(block))
        print(items)
        if(block.startswith('convolutional') or block.startswith('deconvolutional')):
            filters = int(items.get('filters'))
            size = int(items.get('size'))
            bn = items.get('batch_normalize')
            print('bn:{0}'.format(bn))
            if(bn == '1'):
                print("scal_bias:")
                print(buf[start:start+filters])
                start += filters
                print("scal_weights:")
                print(buf[start:start+filters])
                start += filters
                print("bn_running_mean:")
                print(buf[start:start+filters])
                start += filters
                print("bn_running_var:")
                print(buf[start:start+filters])
                start += filters
            else:
                print('conv_bias:')
                print(buf[start:start+filters])
                start+= filters
                #step = size**2*pre_filters*filters + filters*4
            print("conv_filters:" + str(size**2*pre_filters*filters))
            print(buf[start:start+size**2*pre_filters*filters])
            start+=size**2*pre_filters*filters
            print(start)
            print(len(buf))
            pre_filters = filters
            if(block == 'convolutional_17'):
                pre_filters = 256
            '''
            if(block == 'convolutional_20'):
                pre_filters = 384
            ''' #for no ncs
            if(block == 'deconvolutional_21'):
                pre_filters = 384

['net_1', 'convolutional_2', 'maxpool_3', 'convolutional_4', 'maxpool_5', 'convolutional_6', 'maxpool_7', 'convolutional_8', 'maxpool_9', 'convolutional_10', 'maxpool_11', 'convolutional_12', 'maxpool_13', 'convolutional_14', 'convolutional_15', 'convolutional_16', 'convolutional_17', 'yolo_18', 'route_19', 'convolutional_20', 'deconvolutional_21', 'route_22', 'convolutional_23', 'convolutional_24', 'yolo_25']
[     0      2      0 960000      0]
net_1
{'hue': '.1', 'saturation': '1.5', 'angle': '0', 'decay': '0.0005', 'learning_rate': '0.001', 'scales': '.1,.1', 'batch': '1', 'height': '416', 'channels': '3', 'width': '416', 'subdivisions': '1', 'burn_in': '1000', 'policy': 'steps', 'max_batches': '500200', 'steps': '400000,450000', 'momentum': '0.9', 'exposure': '1.5'}
convolutional_2
{'activation': 'leaky', 'stride': '1', 'pad': '1', 'filters': '16', 'batch_normalize': '1', 'size': '3'}
bn:1
scal_bias:
[ 0.19144782  0.40064552 -0.11537587  0.09177608 -1.41123414 -0.56269848
 -0.0144